Giving Geotag (Coordinates)

In [1]:
import os
import piexif
from PIL import Image
import random

def coordinate_to_tuple(coordinate):
        degrees = int(abs(coordinate))
        minutes_float = (abs(coordinate) - degrees) * 60
        minutes = int(minutes_float)
        seconds_float = (minutes_float - minutes) * 60
        seconds = int(abs(seconds_float) * 100)  # Ensure seconds remain positive
        return ((degrees, 1), (minutes, 1), (seconds, 100))

def generate_random_coordinate(latitude, longitude, radius):
    # Generate random coordinates within the specified radius
    latitude_offset = random.uniform(-radius, radius)
    longitude_offset = random.uniform(-radius, radius)
    
    new_latitude = latitude + latitude_offset
    new_longitude = longitude + longitude_offset
    
    return new_latitude, new_longitude

def update_exif_gps_single(image_name_location, latitude, longitude, radius):
    image = Image.open(image_name_location)

    # Generate random coordinates within the specified radius
    new_latitude, new_longitude = generate_random_coordinate(latitude, longitude, radius)
    
    # Prepare GPS IFD data
    latitude_coo = coordinate_to_tuple(new_latitude)
    longitude_coo = coordinate_to_tuple(new_longitude)
    gps_ifd = {
        piexif.GPSIFD.GPSVersionID: (2, 0, 0, 0),
        piexif.GPSIFD.GPSAltitudeRef: 1,
        piexif.GPSIFD.GPSLatitude: latitude_coo,
        piexif.GPSIFD.GPSLatitudeRef: b'N' if new_latitude >= 0 else b'S',  # Ensure bytes type
        piexif.GPSIFD.GPSLongitude: longitude_coo,
        piexif.GPSIFD.GPSLongitudeRef: b'E' if new_longitude >= 0 else b'W',  # Ensure bytes type
    }

    # Load existing Exif data (if any)
    exif_dict = piexif.load(image_name_location)

    # Update GPS information
    exif_dict['GPS'] = gps_ifd

    # Convert Exif to bytes
    exif_bytes = piexif.dump(exif_dict)

    # Save the modified image with updated Exif data
    image.save(image_name_location, exif=exif_bytes)
    

def update_exif_gps_directory(directory, latitude, longitude, radius):
    # Iterate through image files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            image_name_location = os.path.join(directory, filename)
            update_exif_gps_single(image_name_location, latitude, longitude, radius)
            print(filename, "berhasil disimpan")


In [2]:
radius = 0.0001  # You can adjust the radius as needed to control the range of random coordinates

image_directory = "C:/Users/ramad/OneDrive/Dokumen/BAN/OC Rote Ndao/no tag/oelodo"
# latitude =-9.51754027
# longitude =119.0248174

# update_exif_gps_directory(image_directory,-10.68550195,123.29792855, radius)

In [7]:
image_path = "./img/foto_proses_50_bondowoso.jpg"

update_exif_gps_single(image_path,-7.89315249,113.81818787, radius)


Check exif data of image / metadata

In [3]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS


def get_exif(filename):
    exif_data = {}
    image = Image.open(filename)
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPS":
                gps_data = {}
                for gps_tag in value:
                    sub_decoded = GPSTAGS.get(gps_tag, gps_tag) 
                    gps_data[sub_decoded] = value[gps_tag]
                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value

    return exif_data    


In [4]:
if __name__ == "__main__":
    # exif = get_exif("C:/Users/ramad/OneDrive/Dokumen/BAN/OC Sumba Timur/DONE/perdesa/Hadakamali/1203.jpg")
    # exif = get_exif("C:/Users/ramad/OneDrive/Dokumen/BAN/OC Sumba Timur/DONE/perdesa/Ds.Persia.Praikaruku/1116.jpg")
    exif = get_exif("C:/Users/ramad/OneDrive/Dokumen/BAN/OC Sumba Timur/DONE/perdesa/Ananjaki/869.jpg")
    print(exif)
    print("/n")
    exif = get_exif("C:/Users/ramad/OneDrive/Dokumen/BAN/OC Sumba Timur/DONE/perdesa/Ananjaki/870.jpg")
    # exif = get_exif("C:/Users/ramad/OneDrive/Gambar/tes2.jpg")
    print(exif)

{'ImageWidth': 600, 'ImageLength': 800, 'BitsPerSample': (8, 8, 8), 'GPSInfo': {0: b'\x02\x00\x00\x00', 1: 'S', 2: (10.0, 11.0, 48.2), 3: 'E', 4: (120.0, 12.0, 56.04), 5: b'\x01'}, 'PhotometricInterpretation': 2, 'ResolutionUnit': 2, 'ExifOffset': 243, 'Software': 'Adobe Photoshop 21.2 (Windows)', 'Orientation': 1, 'DateTime': '2023:07:27 11:29:30', 'SamplesPerPixel': 3, 'XResolution': 72.0, 'YResolution': 72.0, 'ExifVersion': b'0231', 'ColorSpace': 65535, 'ExifImageWidth': 600, 'ExifImageHeight': 800}
/n
{'ImageWidth': 600, 'ImageLength': 800, 'BitsPerSample': (8, 8, 8), 'GPSInfo': {0: b'\x02\x00\x00\x00', 1: 'S', 2: (10.0, 11.0, 48.59), 3: 'E', 4: (120.0, 12.0, 55.67), 5: b'\x01'}, 'PhotometricInterpretation': 2, 'ResolutionUnit': 2, 'ExifOffset': 243, 'Software': 'Adobe Photoshop 21.2 (Windows)', 'Orientation': 1, 'DateTime': '2023:07:27 11:30:19', 'SamplesPerPixel': 3, 'XResolution': 72.0, 'YResolution': 72.0, 'ExifVersion': b'0231', 'ColorSpace': 65535, 'ExifImageWidth': 600, 'Exi

CHECK IMAGE DIR if has gps or not

In [1]:
import os
import shutil

from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif(filename):
    exif_data = {}
    image = Image.open(filename)
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPS":
                gps_data = {}
                for gps_tag in value:
                    sub_decoded = GPSTAGS.get(gps_tag, gps_tag) 
                    gps_data[sub_decoded] = value[gps_tag]
                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value
    return exif_data

def has_geotag(filename):
    exif_data = get_exif(filename)
    if "GPSInfo" in exif_data:
        return True
    return False

def copy_images_without_geotag(src_directory, dest_directory):
    image_files = [f for f in os.listdir(src_directory) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
    for image_file in image_files:
        image_path = os.path.join(src_directory, image_file)
        if not has_geotag(image_path):
            dest_path = os.path.join(dest_directory, image_file)
            shutil.copy2(image_path, dest_path)
            print(f"{image_file} disalin ke direktori tujuan karena tidak memiliki geotag koordinat.")

# Ganti 'src_direktori' dengan direktori sumber dan 'dest_direktori' dengan direktori tujuan
src_directory = 'C:/Users/ramad/OneDrive/Dokumen/BAN/OC Bondowoso'
dest_directory = 'C:/Users/ramad/OneDrive/Dokumen/BAN/OC Bondowoso/no tag'

copy_images_without_geotag(src_directory, dest_directory)




793 OC Tani Rejo.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
794 OC Fajar Tani.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
795 OC Tani Sejahtera.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
796 OC Darussholah.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
797 OC Podo Makmur.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
798 OC Harapan.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
799 OC Pelita Jaya.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
800 OC Panca Karya.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
801 OC Sumber Rejeki.jpg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
802 OC Gema Tani.jpeg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
803 OC Setia Kawan.jpeg disalin ke direktori tujuan karena tidak memiliki geotag koordinat.
8